# SLAM tutorial

SLAM and factor graphs tutorial prepared for the ORIentate Seminars

Author: Matias Mattamala (matias@robots.ox.ac.uk, [mmattamala@github](https://github.com/mmattamala))

Date: 23/11/2023


> ⚠️ **Warning**: Before proceeding, make sure you installed all the required dependencies in the [`requirements.txt`](../requirements.txt) file!


## Preliminaries

In [ ]:
%pip install -e ../

In [ ]:
import numpy as np
import open3d as o3d

import slam_tutorial
import slam_tutorial.visualization as vis
import slam_tutorial.pose_graph as pg
from slam_tutorial.io import load_pose_graph

## Part 1: Inspecting a perfect SLAM graph
In this first part we will check how a pose graph looks like. For that, we will use some data avaialble in the [assets](../assets/) folder.

In [ ]:
graph = load_pose_graph(
    slam_tutorial.ASSETS_DIR + "/ground_truth.slam",
    clouds_path=slam_tutorial.ASSETS_DIR + "/individual_clouds",
)

The `graph` object stores the **nodes** and **edges** of the pose graph.

Each **node** is defined by:
- A _pose_ $\mathbf{T}_{\mathtt{WB}} \in $ SE(3) ($4\times4$ rigid body matrices), representing the pose of the _base_ in the _world_ frame.
- An _id_
- A _time stamp_ indicating when the node was created.

On the other side, each **edge** is defined by:
- A _parent id_, which indicates what is the origin node for the edge
- A _child id_ representing the node where the edge lands
- A _relative pose_ $\textbf{T}_{\text{parent, child}}$ indicating the rigid body transformation between from the child frame to the parent frame
- A _type_ indicating if the edge was produced by an odometry system (`odometry`) or a loop candidate (`loop`)

In [ ]:
import time

visualizer = o3d.visualization.Visualizer()
visualizer.create_window("Building a Pose Graph")

# Manually run optimizer
for i in range(graph.size):
    visualizer.clear_geometries()
    geometry = vis.graph_to_geometries(
        graph,
        show_frames=True,
        show_edges=True,
        show_nodes=True,
        show_clouds=True,
        odometry_color=vis.GRAY,
        loop_color=vis.RED,
        up_to_node=i,
    )
    for g in geometry:
        visualizer.add_geometry(g)

    ctr = visualizer.get_view_control()
    ctr.set_front([-0.35, -0.57, 0.7])
    ctr.set_up([0.4, 0.5, 0.6])
    ctr.set_zoom(0.24)
    ctr.set_lookat([1.4, -18, 2.0])

    r = visualizer.get_render_option()
    r.point_size = 1

    for g in geometry:
        visualizer.update_geometry(g)

    visualizer.poll_events()
    visualizer.update_renderer()
    time.sleep(0.01)

visualizer.run()

## Part 2: A more realistic pose graph

The previous pose graph was generated with ground truth poses, which is not usually the case.

In general, the graph will be built using an **odometry estimator**, which provides the pose of the robot in some fixed frame.

The odometry estimator will provide a smooth estimate fo the robot's pose, but it **will very likely drift** over time.

In [ ]:
graph_with_drift = pg.add_odometry_drift(
    graph, noise_per_m=0.1, axis="xy", drift_type="random_walk"
)

geometries = vis.graph_to_geometries(
    graph_with_drift,
    show_frames=True,
    show_edges=True,
    show_nodes=True,
    show_clouds=True,
    odometry_color=vis.GRAY,
    loop_color=vis.RED,
)

o3d.visualization.draw_geometries(
    geometries,
    window_name="Realistic Pose Graph",
    zoom=0.24,
    front=[-0.35, -0.57, 0.7],
    lookat=[1.4, -18, 2.0],
    up=[0.4, 0.5, 0.6],
)

## Part 3: Solving a pose graph with factor graphs

In [ ]:
initial_graph = pg.create_test_pose_graph(
    use_wrong_init=True,
    use_noisy_odom=True,
    use_true_loops=False,
    use_false_loops=False,
    init_noise_per_m=0.1,
    init_drift_axis="xy",
    init_drift_type="random_walk",
    odo_noise_per_m=0.01,
    odo_drift_axis="xyz",
    odo_drift_type="random_walk",
    loop_candidate_id_distance=10,
    true_loop_dist_thr=5,
    false_loop_dist_thr=10,
    true_loop_num=2,
    false_loop_num=10,
    loop_info=1000,
)

geometries = vis.graph_to_geometries(
    initial_graph,
    show_frames=True,
    show_edges=True,
    show_nodes=True,
    show_clouds=True,
    odometry_color=vis.GRAY,
    loop_color=vis.RED,
)
o3d.visualization.draw_geometries(
    geometries,
    window_name="Initial Pose Graph",
    zoom=0.24,
    front=[-0.35, -0.57, 0.7],
    lookat=[1.4, -18, 2.0],
    up=[0.4, 0.5, 0.6],
)

### Prepare factor graph

In [ ]:
import gtsam
import numpy as np

# Create a factor graph container and add factors to it
factor_graph = gtsam.NonlinearFactorGraph()

#### Add prior factor

In [ ]:
PRIOR_NOISE = gtsam.noiseModel.Diagonal.Sigmas(0.001 * np.ones(6))
factor_graph.add(gtsam.PriorFactorPose3(0, initial_graph.get_node_pose(0), PRIOR_NOISE))

#### Add odometry factors

In [ ]:
for e in initial_graph.edges:
    if e["type"] == "odometry":
        ODOMETRY_NOISE = gtsam.noiseModel.Diagonal.Information(e["info"])
        factor_graph.add(
            gtsam.BetweenFactorPose3(
                e["parent_id"], e["child_id"], e["pose"], ODOMETRY_NOISE
            )
        )

#### Initialize state values

In [ ]:
initial_estimate = gtsam.Values()
for i, node in enumerate(initial_graph.nodes):
    initial_estimate.insert(i, graph.get_node_pose(i))

#### Visualize factor graph

In [ ]:
factor_graph.saveGraph("test.dot", initial_estimate)

from graphviz import Source

s = Source.from_file("test.dot")
s.view()

#### Optimize

In [ ]:
# copy graph
import copy

optimized_graph = copy.deepcopy(initial_graph)

# Setup optimizer
parameters = gtsam.GaussNewtonParams()
optimizer = gtsam.GaussNewtonOptimizer(factor_graph, initial_estimate, parameters)

result = optimizer.optimize()
for n, _ in enumerate(optimized_graph.nodes):
    optimized_graph.set_node_pose(n, result.atPose3(n))

#### Visualize solution

In [ ]:
geometries = vis.graph_to_geometries(
    optimized_graph,
    show_frames=True,
    show_edges=True,
    show_nodes=True,
    show_clouds=True,
    odometry_color=vis.GRAY,
    loop_color=vis.RED,
)
o3d.visualization.draw_geometries(
    geometries,
    window_name="Optimized Pose Graph",
    zoom=0.24,
    front=[-0.35, -0.57, 0.7],
    lookat=[1.4, -18, 2.0],
    up=[0.4, 0.5, 0.6],
)

#### Step-by-step optimization

In [ ]:
import copy
import open3d as o3d
import time

# copy graph
optimized_graph = copy.deepcopy(initial_graph)

# Setup optimizer
parameters = gtsam.GaussNewtonParams()
optimizer = gtsam.GaussNewtonOptimizer(factor_graph, initial_estimate, parameters)

visualizer = o3d.visualization.Visualizer()
visualizer.create_window("Step-by-step optimization")

# Manually run optimizer
for i in range(10):
    print(f"iter: {i}")
    visualizer.clear_geometries()
    geometry = vis.graph_to_geometries(
        optimized_graph,
        show_frames=True,
        show_edges=True,
        show_nodes=True,
        show_clouds=True,
        odometry_color=vis.GRAY,
        loop_color=vis.RED,
    )
    for g in geometry:
        visualizer.add_geometry(g)

    ctr = visualizer.get_view_control()
    ctr.set_front([-0.35, -0.57, 0.7])
    ctr.set_up([0.4, 0.5, 0.6])
    ctr.set_zoom(0.24)
    ctr.set_lookat([1.4, -18, 2.0])

    r = visualizer.get_render_option()
    r.point_size = 1

    for g in geometry:
        visualizer.update_geometry(g)

    visualizer.poll_events()
    visualizer.update_renderer()

    # optimizer.iterate()
    # result = optimizer.values()
    # for n, _ in enumerate(optimized_graph.nodes):
    #     optimized_graph.set_node_pose(n, result.atPose3(n))
    time.sleep(1)

visualizer.run()

## Part 4: Adding loops

In [ ]:
initial_graph = pg.create_test_pose_graph(
    use_wrong_init=True,
    use_noisy_odom=True,
    use_true_loops=True,
    use_false_loops=True,
    init_noise_per_m=0.1,
    odo_noise_per_m=0.01,
    loop_candidate_id_distance=10,
    true_loop_dist_thr=5,
    true_loop_num=2,
    false_loop_dist_thr=10,
    false_loop_num=5,
    loop_info=1000,
)

geometries = vis.graph_to_geometries(
    initial_graph,
    show_frames=True,
    show_edges=True,
    show_nodes=True,
    show_clouds=True,
    odometry_color=vis.GRAY,
    loop_color=vis.RED,
)
o3d.visualization.draw_geometries(
    geometries,
    window_name="Initial Pose Graph",
    zoom=0.24,
    front=[-0.35, -0.57, 0.7],
    lookat=[1.4, -18, 2.0],
    up=[0.4, 0.5, 0.6],
)

### Setup factor graph

In [ ]:
import gtsam
import numpy as np

# Create a factor graph container and add factors to it
factor_graph = gtsam.NonlinearFactorGraph()

PRIOR_NOISE = gtsam.noiseModel.Diagonal.Sigmas(0.001 * np.ones(6))
factor_graph.add(gtsam.PriorFactorPose3(0, initial_graph.get_node_pose(0), PRIOR_NOISE))

for e in initial_graph.edges:
    if e["type"] == "odometry":
        ODOMETRY_NOISE = gtsam.noiseModel.Diagonal.Information(e["info"])
        factor_graph.add(
            gtsam.BetweenFactorPose3(
                e["parent_id"], e["child_id"], e["pose"], ODOMETRY_NOISE
            )
        )

### Add loop candidate factors

In [ ]:
for e in initial_graph.edges:
    if e["type"] == "loop":
        LOOP_NOISE = gtsam.noiseModel.Diagonal.Information(1000 * np.eye(6))
        # ROBUST_MODEL = gtsam.noiseModel.Robust.Create(
        #     gtsam.noiseModel.mEstimator.DCS.Create(1.0), LOOP_NOISE
        # )

        factor_graph.add(
            gtsam.BetweenFactorPose3(
                e["parent_id"], e["child_id"], e["pose"], ROBUST_MODEL
            )
        )

### Initialize state values

In [ ]:
initial_estimate = gtsam.Values()
for i, node in enumerate(initial_graph.nodes):
    initial_estimate.insert(i, graph.get_node_pose(i))

### Visualize factor graph

In [ ]:
factor_graph.saveGraph("test.dot", initial_estimate)

from graphviz import Source

s = Source.from_file("test.dot")
s.view()

### Optimize

In [ ]:
optimized_graph = copy.deepcopy(initial_graph)

# Setup optimizer
parameters = gtsam.GaussNewtonParams()
optimizer = gtsam.GaussNewtonOptimizer(factor_graph, initial_estimate, parameters)

result = optimizer.optimize()
for n, _ in enumerate(optimized_graph.nodes):
    optimized_graph.set_node_pose(n, result.atPose3(n))

### Visualize solution

In [ ]:
geometries = vis.graph_to_geometries(
    optimized_graph,
    show_frames=True,
    show_edges=True,
    show_nodes=True,
    show_clouds=True,
    odometry_color=vis.GRAY,
    loop_color=vis.RED,
)
o3d.visualization.draw_geometries(
    geometries,
    window_name="Optimized Pose Graph",
    zoom=0.24,
    front=[-0.35, -0.57, 0.7],
    lookat=[1.4, -18, 2.0],
    up=[0.4, 0.5, 0.6],
)